In [16]:
!pip install nltk 

Defaulting to user installation because normal site-packages is not writeable
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 KB 400.4 kB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import  nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import joblib





In [2]:
model_filename = 'Fake_Review_Model_2.joblib'
joblib.dump(model, model_filename)


NameError: name 'model' is not defined

In [3]:
def fit_vectorizer_and_transform(data, vectorizer):
    X = vectorizer.fit_transform(preprocess_reviews(data['text_']))
    return X, vectorizer

def fetch_html_selenium(url):
    try:
        options = Options()
        options.headless = True  # Run Chrome in headless mode

        browser = webdriver.Chrome(options=options)

        browser.get(url)

        WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-text-content"))
        )

        html_content = browser.page_source

        browser.quit()

        return html_content

    except Exception as e:
        print(f"Error fetching HTML with Selenium: {e}")
        return None

def preprocess_reviews(reviews):
    cleaned_reviews = []

    stop_words = set(stopwords.words('english'))

    for review in reviews:
        review = re.sub(r'<.*?>', '', review)
        review = re.sub(r'[^a-zA-Z\s]', '', review)

        review = review.lower()

        words = word_tokenize(review)

        # Remove stopwords
        words = [word for word in words if word not in stop_words]

        cleaned_reviews.append(' '.join(words))

    return cleaned_reviews

def extract_reviews(html):
    soup = BeautifulSoup(html, 'html.parser')

    review_elements = soup.find_all(class_='review-text-content')

    reviews = [element.get_text(strip=True) for element in review_elements]

    return reviews

if __name__ == "__main__":
    #data = pd.read_csv('fake_reveiw_train.csv')  

    model = joblib.load('Fake_Review_Model_hopeful.joblib')
    vectorizer = joblib.load('vectorizer_svc.joblib')

    #X_train, fitted_vectorizer = fit_vectorizer_and_transform(data, vectorizer)

    #y_train = data['label']  
    #model.fit(X_train, y_train)

    target_url = 'https://www.amazon.in/Sennheiser-Wireless-350BT-Black-Headphone/dp/B083MNJ2VX/ref=sr_1_2_sspa?keywords=headphones&s=electronics&sr=1-2-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&th=1'

    # Fetch HTML content using Selenium
    html_content = fetch_html_selenium(target_url)

    if html_content:
        print("Successfully fetched HTML.")

        reviews = extract_reviews(html_content)

        cleaned_reviews = preprocess_reviews(reviews)

        X_test = vectorizer.transform(cleaned_reviews)

        # Transform the reviews using the fitted vectorizer
        
        predictions = model.predict(X_test)

        for review, prediction in zip(reviews, predictions):
            print(f"Review:\n{review}\nPrediction: {'Fake' if prediction == 1 else 'Real'}\n")

    else:
        print("Failed to fetch HTML.")

Successfully fetched HTML.
Review:
It's is really good headphone. The built quility is good and premium, earpads are very soft but bit small in size but it will comfortable in 2 days, connectivity is very good, good for movies and music (not bass), battery life is also very good. At first I didn't like this headphones because it has low sound with compare to my previous earphone but after using this headphone for 2 days I loved it. It has balanced sound not for bass lovers. Vocals are very clear and you can differentiate between vocals and instruments easily. If you love more vocal than bass than go for it.
Prediction: Real

Review:
Here's the TLDR: gets uncomfortable after about an hour of use if you have big ears like I do. Sounds pretty good considering the price, slightly on the bassier side, but fairly neutral overall. Best in the price range I think, at least compared to the offerings from JBL. The marshall major 3 wireless has better sound, build quality and comfort but doesn't 